In [36]:
import pandas as pd
import subprocess
import re
import os
import matplotlib.pyplot as plt
import shutil
from multiprocessing import Pool
from pathlib import Path
import shutil

In [11]:
#!!TODO: возможно надо делать меньше порог на p-value

In [37]:
d = pd.read_csv("../all_marks/ChiP-Seq_for_all_marks_all_replicas.tsv", sep='\t', index_col=False)

In [19]:
#macs2 filterdup -i CTCF_ChIP_200K.bed.gz --keep-dup=1 -o CTCF_ChIP_200K_filterdup.bed
#macs2 filterdup -i CTCF_ChIP_200K_control.bed.gz --keep-dup=1 -o CTCF_ChIP_200K_control_filterdup.bed
#macs2 predictd -i CTCF_ChIP_200K_filterdup.bed -g hs -m 5 50
#macs2 pileup -i CTCF_ChIP_200K_filterdup.bed -o CTCF_ChIP_200K_filterdup.pileup.bdg --extsize 254
#macs2 pileup -i CTCF_Control_200K_filterdup.bed -B --extsize 127 -o d_bg.bdg
#macs2 pileup -i CTCF_Control_200K_filterdup.bed -B --extsize 500 -o 1k_bg.bdg
#macs2 bdgopt -i 1k_bg.bdg -m multiply -p 0.254 -o 1k_bg_norm.bdg
#macs2 pileup -i CTCF_Control_200K_filterdup.bed -B --extsize 5000 -o 10k_bg.bdg
#macs2 bdgopt -i 1k_bg.bdg -m multiply -p 0.0254 -o 1k_bg_norm.bdg
#macs2 bdgcmp -m max -t 1k_bg_norm.bdg -c 10k_bg_norm.bdg -o 1k_10k_bg_norm.bdg
#macs2 bdgcmp -m max -t 1k_10k_bg_norm.bdg -c d_bg.bdg -o d_1k_10k_bg_norm.bdg
#macs2 bdgopt -i d_1k_10k_bg_norm.bdg -m max -p .0188023 -o local_bias_raw.bdg
#macs2 bdgopt -i local_bias_raw.bdg -m multiply -p .99858 -o local_lambda.bdg
#macs2 bdgcmp -t CTCF_ChIP_200K_filterdup.pileup.bdg -c local_lambda.bdg -m qpois -o CTCF_ChIP_200K_qvalue.bdg
#macs2 bdgbroadcall -i CTCF_ChIP_200K_qvalue.bdg -c 1.301 -l 245 -g 100 -o CTCF_ChIP_200K_peaks.bed
#or& for narrow peaks:
#macs2 bdgpeakcall -i CTCF_ChIP_200K_qvalue.bdg -c 1.301 -l 245 -g 100 -o CTCF_ChIP_200K_peaks.bed
def callPeaks(data):
    signal = data[0]
    signal_path = data[1]
    control = data[2]
    control_path = data[3]
    norm_factor = data[4]
    tmp_dir = "/data/mazurovev/tmp_peaks/tmp_call_peaks_for_" + signal
    os.mkdir(tmp_dir)
    print(signal)
    try:
        output = subprocess.check_output("macs2 filterdup -i " + signal_path + signal + ".bed" + " --keep-dup=1 -o " + tmp_dir + "/" + signal + "_filterdup.bed", shell=True, stderr=subprocess.STDOUT).decode()
        m = re.match(r".+tags after filtering in alignment file: (\d+).+", output, re.MULTILINE|re.DOTALL)
        signal_reads_count = m.group(1)
        
        output = subprocess.check_output("macs2 filterdup -i " + control_path + control + ".bed" + " --keep-dup=1 -o " + tmp_dir + "/" + control + "_filterdup.bed", shell=True, stderr=subprocess.STDOUT).decode()
        m = re.match(r".+tags after filtering in alignment file: (\d+).+", output, re.MULTILINE|re.DOTALL)
        control_reads_count = m.group(1)
        
        output = subprocess.check_output("macs2 predictd -i " + tmp_dir + "/" + signal + "_filterdup.bed" + " -g hs -m 5 50", shell=True, stderr=subprocess.STDOUT).decode()
        m = re.match(r".+predicted fragment length is (\d+).+", output, re.MULTILINE|re.DOTALL)
        extsize = m.group(1)
        
        output = subprocess.check_output("macs2 pileup -i " + tmp_dir + "/" + signal + "_filterdup.bed" + " -f BED -o " + tmp_dir + "/" + signal + "_filterdup.pileup.bdg --extsize " + extsize, shell=True, stderr=subprocess.STDOUT).decode()
        
        output = subprocess.check_output("macs2 pileup -i " + tmp_dir + "/" + control + "_filterdup.bed" + " -B -f BED --extsize " + str(int(extsize)//2) + " -o " + tmp_dir + "/" + control + "_d_bg.bdg", shell=True, stderr=subprocess.STDOUT).decode()
        
        output = subprocess.check_output("macs2 pileup -i " + tmp_dir + "/" + control + "_filterdup.bed" + " -B -f BED --extsize 500 -o " + tmp_dir + "/" + control + "_1k_bg.bdg", shell=True, stderr=subprocess.STDOUT).decode()
        
        output = subprocess.check_output("macs2 bdgopt -i " + tmp_dir + "/" + control + "_1k_bg.bdg" + " -m multiply -p " + str(int(extsize)/1000) + " -o " + tmp_dir + "/" + control + "_1k_bg_norm.bdg", shell=True, stderr=subprocess.STDOUT).decode()
        
        output = subprocess.check_output("macs2 pileup -i " + tmp_dir + "/" + control + "_filterdup.bed" + " -B -f BED --extsize 5000 -o " + tmp_dir + "/" + control + "_10k_bg.bdg", shell=True, stderr=subprocess.STDOUT).decode()
        
        output = subprocess.check_output("macs2 bdgopt -i " + tmp_dir + "/" + control + "_10k_bg.bdg" + " -m multiply -p " + str(int(extsize)/10000) + " -o " + tmp_dir + "/" + control + "_10k_bg_norm.bdg", shell=True, stderr=subprocess.STDOUT).decode()
        
        genome_bg = int(control_reads_count)*int(extsize)/2900000000
        
        output = subprocess.check_output("macs2 bdgcmp -m max -t " + tmp_dir + "/" + control + "_1k_bg_norm.bdg" + " -c " + tmp_dir + "/" + control + "_10k_bg_norm.bdg" + " -o " + tmp_dir + "/" + control + "_1k_10k_bg_norm.bdg", shell=True, stderr=subprocess.STDOUT).decode()
        
        output = subprocess.check_output("macs2 bdgcmp -m max -t " + tmp_dir + "/" + control + "_1k_10k_bg_norm.bdg" + " -c " + tmp_dir + "/" + control + "_d_bg.bdg" + " -o " + tmp_dir + "/" + control + "_d_1k_10k_bg_norm.bdg", shell=True, stderr=subprocess.STDOUT).decode()
        
        output = subprocess.check_output("macs2 bdgopt -i " + tmp_dir + "/" + control + "_d_1k_10k_bg_norm.bdg" + " -m max -p " + str(genome_bg) + " -o " + tmp_dir + "/" + control + "_local_bias_raw.bdg", shell=True, stderr=subprocess.STDOUT).decode()
        
        output = subprocess.check_output("macs2 bdgopt -i " + tmp_dir + "/" + control + "_local_bias_raw.bdg" + " -m multiply -p " + str(norm_factor) + " -o " + tmp_dir + "/" + control + "_local_lambda.bdg", shell=True, stderr=subprocess.STDOUT).decode()
        
        output = subprocess.check_output("macs2 bdgcmp -t " + tmp_dir + "/" + signal + "_filterdup.pileup.bdg" + " -c " + tmp_dir + "/" + control + "_local_lambda.bdg -m qpois -p 1" + " -o " + tmp_dir + "/" + signal + "_qvalue.bdg", shell=True, stderr=subprocess.STDOUT).decode()
        
        #broad or narrow peaks
        #output = subprocess.check_output("macs2 bdgbroadcall -i " + tmp_dir + "/" + signal + "_qvalue.bdg" + " -c 1.301 " + " -o " + signal_path + signal + "_peaks.bed", shell=True, stderr=subprocess.STDOUT).decode()
        output = subprocess.check_output("macs2 bdgpeakcall -i " + tmp_dir + "/" + signal + "_qvalue.bdg" + " -c 1.301 -l " + extsize + " -o " + signal_path + signal + "_narrow_peaks.bed", shell=True, stderr=subprocess.STDOUT).decode()
        print("done " + signal)
    except subprocess.CalledProcessError as e:
        output = e.output.decode()
        
    #shutil.rmtree(tmp_dir)
    return output

In [44]:
#TODO: попробовать другой broad_cutoff?
def call_peaks_macs2(data):
    signal = data[0]
    signal_path = data[1]
    control = data[2]
    control_path = data[3]
    norm_factor = data[4]
    print("macs2 callpeak --outdir " + signal_path + " -t " + signal_path + signal + ".bed" + 
                                         " -c " + control_path + control + ".bed" + 
                                         " -f BED --broad --broad-cutoff 0.1 -g 2.9e9 -q 0.05 -n " + signal + " --ratio " + str(norm_factor))
    try:
        output = subprocess.check_output("macs2 callpeak --outdir " + signal_path + " -t " + signal_path + signal + ".bed" + 
                                         " -c " + control_path + control + ".bed" + 
                                         " -f BED --broad --broad-cutoff 0.1 -g 2.9e9 -q 0.05 -n " + signal + " --ratio " + str(norm_factor), shell=True, stderr=subprocess.STDOUT).decode()
    except subprocess.CalledProcessError as e:
        output = e.output.decode()
        
    #shutil.rmtree(tmp_dir)
    return output   

In [41]:
data = []
for index, row in d.iterrows():
    if(row['Experiment target'] == 'H3K27me3-human'):
        signal_path = "/data/mazurovev/" + row['Experiment target'].split('-')[0] + "/" + row['Biosample term id'] + "/"
        control_path = "/data/mazurovev/controls/"
        data.append((row['File accession'], signal_path, row['Control'], control_path, row['norm_factor']))

In [45]:
pool = Pool(processes=10)
outputs = pool.map(call_peaks_macs2, data)

pool.close()
pool.join()

macs2 callpeak --outdir /data/mazurovev/H3K27me3/UBERON:0002107/ -t /data/mazurovev/H3K27me3/UBERON:0002107/ENCFF457HGI.bed -c /data/mazurovev/controls/ENCFF964MSO.bed -f BED --broad --broad-cutoff 0.1 -g 2.9e9 -q 0.05 -n ENCFF457HGI --ratio 0.761293
macs2 callpeak --outdir /data/mazurovev/H3K27me3/CL:0002327/ -t /data/mazurovev/H3K27me3/CL:0002327/ENCFF015OVW.bed -c /data/mazurovev/controls/ENCFF801RTD.bed -f BED --broad --broad-cutoff 0.1 -g 2.9e9 -q 0.05 -n ENCFF015OVW --ratio 0.3841786
macs2 callpeak --outdir /data/mazurovev/H3K27me3/UBERON:0002369/ -t /data/mazurovev/H3K27me3/UBERON:0002369/ENCFF546HYL.bed -c /data/mazurovev/controls/ENCFF237BVB.bed -f BED --broad --broad-cutoff 0.1 -g 2.9e9 -q 0.05 -n ENCFF546HYL --ratio 0.8439027
macs2 callpeak --outdir /data/mazurovev/H3K27me3/UBERON:0000992/ -t /data/mazurovev/H3K27me3/UBERON:0000992/ENCFF186TYR.bed -c /data/mazurovev/controls/ENCFF791PRR.bed -f BED --broad --broad-cutoff 0.1 -g 2.9e9 -q 0.05 -n ENCFF186TYR --ratio -1.0
macs2 

macs2 callpeak --outdir /data/mazurovev/H3K27me3/UBERON:0004648/ -t /data/mazurovev/H3K27me3/UBERON:0004648/ENCFF131KLD.bed -c /data/mazurovev/controls/ENCFF255WXC.bed -f BED --broad --broad-cutoff 0.1 -g 2.9e9 -q 0.05 -n ENCFF131KLD --ratio 0.4564688
macs2 callpeak --outdir /data/mazurovev/H3K27me3/CL:1001606/ -t /data/mazurovev/H3K27me3/CL:1001606/ENCFF020YIZ.bed -c /data/mazurovev/controls/ENCFF178GZR.bed -f BED --broad --broad-cutoff 0.1 -g 2.9e9 -q 0.05 -n ENCFF020YIZ --ratio 1.529778
macs2 callpeak --outdir /data/mazurovev/H3K27me3/UBERON:0002046/ -t /data/mazurovev/H3K27me3/UBERON:0002046/ENCFF249BFZ.bed -c /data/mazurovev/controls/ENCFF464KVS.bed -f BED --broad --broad-cutoff 0.1 -g 2.9e9 -q 0.05 -n ENCFF249BFZ --ratio 1.117332
macs2 callpeak --outdir /data/mazurovev/H3K27me3/UBERON:0011907/ -t /data/mazurovev/H3K27me3/UBERON:0011907/ENCFF650KVU.bed -c /data/mazurovev/controls/ENCFF287TUT.bed -f BED --broad --broad-cutoff 0.1 -g 2.9e9 -q 0.05 -n ENCFF650KVU --ratio 0.5857286
ma

macs2 callpeak --outdir /data/mazurovev/H3K27me3/UBERON:0002369/ -t /data/mazurovev/H3K27me3/UBERON:0002369/ENCFF732UZB.bed -c /data/mazurovev/controls/ENCFF962WPS.bed -f BED --broad --broad-cutoff 0.1 -g 2.9e9 -q 0.05 -n ENCFF732UZB --ratio 0.38225
macs2 callpeak --outdir /data/mazurovev/H3K27me3/EFO:0003042/ -t /data/mazurovev/H3K27me3/EFO:0003042/ENCFF830PVE.bed -c /data/mazurovev/controls/ENCFF236MNB.bed -f BED --broad --broad-cutoff 0.1 -g 2.9e9 -q 0.05 -n ENCFF830PVE --ratio 0.5540645000000001
macs2 callpeak --outdir /data/mazurovev/H3K27me3/CL:0000746/ -t /data/mazurovev/H3K27me3/CL:0000746/ENCFF407XPY.bed -c /data/mazurovev/controls/ENCFF120GPO.bed -f BED --broad --broad-cutoff 0.1 -g 2.9e9 -q 0.05 -n ENCFF407XPY --ratio 1.028216
macs2 callpeak --outdir /data/mazurovev/H3K27me3/CL:0000515/ -t /data/mazurovev/H3K27me3/CL:0000515/ENCFF462BRB.bed -c /data/mazurovev/controls/ENCFF298NLV.bed -f BED --broad --broad-cutoff 0.1 -g 2.9e9 -q 0.05 -n ENCFF462BRB --ratio 1.533635
macs2 cal

macs2 callpeak --outdir /data/mazurovev/H3K27me3/EFO:0002784/ -t /data/mazurovev/H3K27me3/EFO:0002784/ENCFF175YYN.bed -c /data/mazurovev/controls/ENCFF684NLR.bed -f BED --broad --broad-cutoff 0.1 -g 2.9e9 -q 0.05 -n ENCFF175YYN --ratio 0.8666805000000001
macs2 callpeak --outdir /data/mazurovev/H3K27me3/CL:0002328/ -t /data/mazurovev/H3K27me3/CL:0002328/ENCFF518FAE.bed -c /data/mazurovev/controls/ENCFF682GMA.bed -f BED --broad --broad-cutoff 0.1 -g 2.9e9 -q 0.05 -n ENCFF518FAE --ratio 0.7422186
macs2 callpeak --outdir /data/mazurovev/H3K27me3/UBERON:0008952/ -t /data/mazurovev/H3K27me3/UBERON:0008952/ENCFF316LIQ.bed -c /data/mazurovev/controls/ENCFF163YFX.bed -f BED --broad --broad-cutoff 0.1 -g 2.9e9 -q 0.05 -n ENCFF316LIQ --ratio 1.367473
macs2 callpeak --outdir /data/mazurovev/H3K27me3/CL:0002327/ -t /data/mazurovev/H3K27me3/CL:0002327/ENCFF520NCF.bed -c /data/mazurovev/controls/ENCFF163IHR.bed -f BED --broad --broad-cutoff 0.1 -g 2.9e9 -q 0.05 -n ENCFF520NCF --ratio 0.3776231
macs2 

In [ ]:
#========================

In [ ]:
#callpeak -t ENCFF318IMH.bed -c /data/mazurovev/controls/ENCFF535QXZ.bed -f BED --broad -g 2.9e9 -q 0.05 -n macs2_original --broad-cutoff 0.05 --ratio 0.1739588